In [ ]:
%tensorflow_version 1.x


TensorFlow 1.x selected.


In [ ]:
!pip install gluoncv

     |████████████████████████████████| 1.1MB 8.3MB/s 
     |████████████████████████████████| 337kB 22.9MB/s 
     |████████████████████████████████| 14.1MB 323kB/s 
     |████████████████████████████████| 122kB 38.4MB/s 
     |████████████████████████████████| 952kB 36.5MB/s 
  Installing build dependencies ... canceled
ERROR: Operation cancelled by user


In [ ]:
!pip install mxnet

     |████████████████████████████████| 46.9MB 96kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [ ]:
!pip install gluoncv --upgrade

Requirement already up-to-date: gluoncv in /usr/local/lib/python3.7/dist-packages (0.10.1.post0)


In [ ]:
from matplotlib import pyplot as plt
from gluoncv import model_zoo, data, utils
import numpy as np


In [ ]:
rcnn = model_zoo.get_model('faster_rcnn_resnet50_v1b_voc', pretrained=True)


121888KB [00:03, 31830.76KB/s]                            


our trial

In [ ]:
import numpy as np
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from itertools import islice

In [ ]:
import json

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class ROIPoolingLayer(Layer):
    """ Implements Region Of Interest Max Pooling 
        for channel-first images and relative bounding box coordinates
        
        # Constructor parameters
            pooled_height, pooled_width (int) -- 
              specify height and width of layer outputs
        
        Shape of inputs
            [(batch_size, pooled_height, pooled_width, n_channels),
             (batch_size, num_rois, 4)]
           
        Shape of output
            (batch_size, num_rois, pooled_height, pooled_width, n_channels)
    
    """
    def __init__(self, pooled_height, pooled_width, **kwargs):
        self.pooled_height = pooled_height
        self.pooled_width = pooled_width
        
        super(ROIPoolingLayer, self).__init__(**kwargs)
        
    def compute_output_shape(self, input_shape):
        """ Returns the shape of the ROI Layer output
        """
        feature_map_shape, rois_shape = input_shape
        assert feature_map_shape[0] == rois_shape[0]
        batch_size = feature_map_shape[0]
        n_rois = rois_shape[1]
        n_channels = feature_map_shape[3]
        return (batch_size, n_rois, self.pooled_height, 
                self.pooled_width, n_channels)

    def call(self, x):
        """ Maps the input tensor of the ROI layer to its output
        
            # Parameters
                x[0] -- Convolutional feature map tensor,
                        shape (batch_size, pooled_height, pooled_width, n_channels)
                x[1] -- Tensor of region of interests from candidate bounding boxes,
                        shape (batch_size, num_rois, 4)
                        Each region of interest is defined by four relative 
                        coordinates (x_min, y_min, x_max, y_max) between 0 and 1
            # Output
                pooled_areas -- Tensor with the pooled region of interest, shape
                    (batch_size, num_rois, pooled_height, pooled_width, n_channels)
        """
        def curried_pool_rois(x): 
          return ROIPoolingLayer._pool_rois(x[0], x[1], 
                                            self.pooled_height, 
                                            self.pooled_width)
        
        pooled_areas = tf.map_fn(curried_pool_rois, x, dtype=tf.float32)

        return pooled_areas
    
    @staticmethod
    def _pool_rois(feature_map, rois, pooled_height, pooled_width):
        """ Applies ROI pooling for a single image and varios ROIs
        """
        def curried_pool_roi(roi): 
          return ROIPoolingLayer._pool_roi(feature_map, roi, 
                                           pooled_height, pooled_width)
        
        pooled_areas = tf.map_fn(curried_pool_roi, rois, dtype=tf.float32)
        return pooled_areas
    
    @staticmethod
    def _pool_roi(feature_map, roi, pooled_height, pooled_width):
        """ Applies ROI pooling to a single image and a single region of interest
        """

        # Compute the region of interest        
        feature_map_height = int(feature_map.shape[0])
        feature_map_width  = int(feature_map.shape[1])
        
        h_start = tf.cast(feature_map_height * roi[0], 'int32')
        w_start = tf.cast(feature_map_width  * roi[1], 'int32')
        h_end   = tf.cast(feature_map_height * roi[2], 'int32')
        w_end   = tf.cast(feature_map_width  * roi[3], 'int32')
        
        region = feature_map[h_start:h_end, w_start:w_end, :]
        
        # Divide the region into non overlapping areas
        region_height = h_end - h_start
        region_width  = w_end - w_start
        h_step = tf.cast( region_height / pooled_height, 'int32')
        w_step = tf.cast( region_width  / pooled_width , 'int32')
        
        areas = [[(
                    i*h_step, 
                    j*w_step, 
                    (i+1)*h_step if i+1 < pooled_height else region_height, 
                    (j+1)*w_step if j+1 < pooled_width else region_width
                   ) 
                   for j in range(pooled_width)] 
                  for i in range(pooled_height)]
        
        # take the maximum of each area and stack the result
        def pool_area(x): 
          return tf.math.reduce_max(region[x[0]:x[2], x[1]:x[3], :], axis=[0,1])
        
        pooled_features = tf.stack([[pool_area(x) for x in row] for row in areas])
        return pooled_features

In [ ]:
import pickle


with open('/content/gdrive/MyDrive/btp_data/im_name_feature_rcnn.pkl', 'rb') as f:
    im_name_feature_rcnn= pickle.load(f)

In [ ]:
im_cap = '/content/gdrive/MyDrive/btp_data/im_capno_400.json'
with open(im_cap) as train_file1:
    im_cap_no= json.load(train_file1)

In [ ]:
print(len(im_cap_no))

1609


In [ ]:
batch_size = 400
img_height = 28
img_width = 28
n_channels = 512
n_rois = 3
pooled_height = 7
pooled_width = 7

In [ ]:
args = []
for k,v in im_name_feature_rcnn.items():
  args.append(v)
batch_im_feature = np.concatenate(args)  #1609*28*28*512

fail trial


In [ ]:
'''arg_roi=[]
for i,(k,v) in enumerate(im_cap_no.items()):
  print(i)
  im_fname='/content/gdrive/MyDrive/Use Image/'+str(k)+'_'+str(v) +'.jpg'
  x, orig_img = data.transforms.presets.rcnn.load_test(im_fname, short=224)
  class_IDs, scores, bounding_boxs = rcnn(x)
  forbidden=[]
  arg_roi_in=[]
  for cnt in range(10):
    ma=-1
    mai=0
    for num,itr in enumerate(scores[0]):
      if num not in forbidden:
        if itr>ma:
          ma=itr
          mai=num
    forbidden.append(mai)
    arg_roi_in.append(bounding_boxs[0][mai])
  xt=np.concatenate(arg_roi_in)
  arg_roi.append(xt)
batch_roi=np.concatenate(arg_roi)
'''


"arg_roi=[]\nfor i,(k,v) in enumerate(im_cap_no.items()):\n  print(i)\n  im_fname='/content/gdrive/MyDrive/Use Image/'+str(k)+'_'+str(v) +'.jpg'\n  x, orig_img = data.transforms.presets.rcnn.load_test(im_fname, short=224)\n  class_IDs, scores, bounding_boxs = rcnn(x)\n  forbidden=[]\n  arg_roi_in=[]\n  for cnt in range(10):\n    ma=-1\n    mai=0\n    for num,itr in enumerate(scores[0]):\n      if num not in forbidden:\n        if itr>ma:\n          ma=itr\n          mai=num\n    forbidden.append(mai)\n    arg_roi_in.append(bounding_boxs[0][mai])\n  xt=np.concatenate(arg_roi_in)\n  arg_roi.append(xt)\nbatch_roi=np.concatenate(arg_roi)\n"

In [ ]:
arg_roi=[]  #10*4 
for i,(k,v) in enumerate(im_cap_no.items()):
  print(i)
  im_fname='/content/gdrive/MyDrive/Use Image/'+str(k)+'_'+str(v) +'.jpg'
  x, orig_img = data.transforms.presets.rcnn.load_test(im_fname, short=224)
  class_IDs, scores, bounding_boxs = rcnn(x)
  forbidden=[]
  arg_roi_in=[]  #1 dimens (4)
  #print("size of for: "+str(len(forbidden)))
  for cnt in range(n_rois):
    ma=-1
    mai=0
    for num,itr in enumerate(scores[0]):
      #print(num)
      #print(itr[0])
      if num not in forbidden:
        if itr[0]>ma:
          ma=itr[0]
          mai=num
    forbidden.append(mai)
    ty=bounding_boxs[0][mai].reshape(1,4).asnumpy()
    arg_roi_in.append(ty)
    #print(ty.shape)
    #print(ty)
    #print(type(ty))
    #print(ty.shape)
  #print(arg_roi_in)
  xt=np.concatenate(arg_roi_in,axis=0)
  #print(xt.shape)
  #print(xt)
  xt=np.reshape(xt,(1,n_rois,4))
  arg_roi.append(xt)
  #print(xt.shape)
  #print(xt)
batch_roi=np.concatenate(arg_roi)



In [ ]:
print(batch_roi.shape)

In [ ]:
argu=[]
for i in range(len(im_cap_no)):
  argu.append(np.reshape(batch_im_feature[i],(1,28,28,512)))
batch_im_feature=np.concatenate(argu)
print(batch_im_feature.shape)

In [ ]:
feature_maps_shape = (batch_size, img_height, img_width, n_channels)
feature_maps_tf = tf.placeholder(tf.float32, shape=feature_maps_shape)
roiss_tf = tf.placeholder(tf.float32, shape=(batch_size, n_rois, 4))

In [ ]:
roi_layer = ROIPoolingLayer(pooled_height, pooled_width)
pooled_features = roi_layer([feature_maps_tf, roiss_tf])
print(f"output shape of layer call = {pooled_features.shape}")
# Run tensorflow session
with tf.Session() as session:
    result = session.run(pooled_features, 
                         feed_dict={feature_maps_tf:batch_im_feature,  
                                    roiss_tf:batch_roi})
    
print(f"result.shape = {result.shape}")

In [ ]:
dic_roi={}
for i,(k,v) in enumerate(im_cap_no.items()):
  dic_roi[k]=np.reshape(result[i],(1,75264))


In [ ]:
import pickle
allFiles = "/content/gdrive/MyDrive/btp_data/roi_dic_400.pkl"
open_file = open(allFiles, "wb")
pickle.dump(dic_roi, open_file)
open_file.close()

In [ ]:
#ax = utils.viz.plot_bbox(orig_img, bounding_boxs[0], scores[0], class_IDs[0], class_names=rcnn.classes)
#plt.show()